In [1]:

# import basic modules
import sys
import os
import time
import numpy as np
from tqdm import tqdm
import gc
import torch
import argparse
from scipy.io import loadmat
import h5py
import pandas as pd
import nibabel as nib
# import custom modules
root_dir   = os.path.dirname(os.path.dirname(os.getcwd()))
root_dir = os.path.join(root_dir, 'code')
sys.path.append(os.path.join(root_dir))
from model_src import fwrf_fit, fwrf_predict
from model_fitting import initialize_fitting

# import custom modules
from model_src import rf_grid, gabor_feature_extractor
from utils import file_utility, load_nsd, roi_utils


fpX = np.float32

In [20]:

subject = 1
# device = initialize_fitting.init_cuda()
device = torch.device('cpu:0')
nsd_root, stim_root, beta_root, mask_root = initialize_fitting.get_paths()
voxel_mask, voxel_idx, [roi_labels_retino, roi_labels_categ], ncsnr_full, brain_nii_shape = get_voxel_info(mask_root, beta_root, subject)

3794 voxels of overlap between kastner and prf definitions, using prf defs
unique values in retino labels:
[-1.  0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16.
 17. 18. 19. 20. 21. 22. 23. 24. 25.]
0 voxels of overlap between face and place definitions, using place defs
unique values in categ labels:
[-1.  0. 26. 27. 28. 30. 31. 32. 33.]
1535 voxels are defined (differently) in both retinotopic areas and category areas

14913 voxels are defined across all areas, and will be used for analysis

Loading numerical label/name mappings for all ROIs:
[1, 2, 3, 4, 5, 6, 7]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
['V1v', 'V1d', 'V2v', 'V2d', 'V3v', 'V3d', 'hV4', 'VO1', 'VO2', 'PHC1', 'PHC2', 'TO2', 'TO1', 'LO2', 'LO1', 'V3B', 'V3A', 'IPS0', 'IPS1', 'IPS2', 'IPS3', 'IPS4', 'IPS5', 'SPL1', 'FEF']
[1, 2, 3, 4, 5]
['OFA', 'FFA-1', 'FFA-2', 'mTL-faces', 'aTL-faces']
[1, 2, 3]
['O

In [26]:
np.sum(roi_labels_retino==25)

72

In [19]:
def get_voxel_info(mask_root, beta_root, subject):

    # first loading each roi definitions file - 3D niftis with diff numbers for each ROI.
    prf_labels_full  = file_utility.load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/prf-visualrois.nii.gz"%subject)
    kast_labels_full = file_utility.load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/Kastner2015.nii.gz"%(subject))
    face_labels_full = file_utility.load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/floc-faces.nii.gz"%(subject))
    place_labels_full = file_utility.load_mask_from_nii(mask_root + "subj%02d/func1pt8mm/roi/floc-places.nii.gz"%(subject))

    # boolean masks of which voxels had definitions in each of these naming schemes
    has_prf_label = (prf_labels_full>0).flatten().astype(bool)
    has_kast_label = (kast_labels_full>0).flatten().astype(bool)
    has_face_label = (face_labels_full>0).flatten().astype(bool)
    has_place_label = (place_labels_full>0).flatten().astype(bool)

    # going to grab these values of ncsnr for each voxel too.
    ncsnr_full = file_utility.load_mask_from_nii(beta_root + "subj%02d/func1pt8mm/betas_fithrf_GLMdenoise_RR/ncsnr.nii.gz"%subject)

    # save the shape, so we can project back to volume space later.
    brain_nii_shape = prf_labels_full.shape

    # To combine all regions, first starting with the kastner atlas for retinotopic ROIs.
    roi_labels_retino = np.copy(kast_labels_full.flatten())
    print('%d voxels of overlap between kastner and prf definitions, using prf defs'%np.sum(has_kast_label & has_prf_label))
    roi_labels_retino[has_prf_label] = prf_labels_full.flatten()[has_prf_label] # overwrite with prf rois
    print('unique values in retino labels:')
    print(np.unique(roi_labels_retino))

    # Next, re-numbering the face/place ROIs so that they have unique numbers not overlapping w retino...
    max_ret_label = np.max(roi_labels_retino)
    face_labels_renumbered = np.copy(face_labels_full.flatten())
    face_labels_renumbered[has_face_label] = face_labels_renumbered[has_face_label] + max_ret_label
    max_face_label = np.max(face_labels_renumbered)
    place_labels_renumbered = np.copy(place_labels_full.flatten())
    place_labels_renumbered[has_place_label] = place_labels_renumbered[has_place_label] + max_face_label

    # Now going to make a separate volume for labels of the category-selective ROIs. 
    # These overlap with the retinotopic defs quite a bit, so want to save both for greater flexibility later on.
    roi_labels_categ = np.copy(face_labels_renumbered.flatten())
    print('%d voxels of overlap between face and place definitions, using place defs'%np.sum(has_face_label & has_place_label))
    roi_labels_categ[has_place_label] = place_labels_renumbered.flatten()[has_place_label] # overwrite with prf rois
    print('unique values in categ labels:')
    print(np.unique(roi_labels_categ))

    # how much overlap between these sets of roi definitions?
    print('%d voxels are defined (differently) in both retinotopic areas and category areas'%np.sum((has_kast_label | has_prf_label) & (has_face_label | has_place_label)))

    voxel_mask = np.logical_or(roi_labels_retino>0, roi_labels_categ>0)
    voxel_idx = np.where(voxel_mask)
    print('\n%d voxels are defined across all areas, and will be used for analysis\n'%np.size(voxel_idx))

    # Now going to print out some more information about these rois and their individual sizes
    print('Loading numerical label/name mappings for all ROIs:')
    ret, face, place = roi_utils.load_roi_label_mapping(nsd_root, subject, verbose=True)

    print('\nSizes of all defined ROIs in this subject:')
    ret_vox_total = 0
    ret_group_names = roi_utils.ret_group_names
    ret_group_inds =  roi_utils.ret_group_inds
#     ret_group_names = ['V1', 'V2', 'V3','hV4','VO1-2','PHC1-2','LO1-2','TO1-2','V3ab','IPS0-5','SPL1','FEF']
#     ret_group_inds = [[1,2],[3,4],[5,6],[7],[8,9],[10,11],[14,15],[12,13],[16,17],[18,19,20,21,22,23],[24],[25]]

    # checking these grouping labels to make sure we have them correct (print which subregions go to which label)
    for gi, group in enumerate(ret_group_inds):
        n_this_region = np.sum(np.isin(roi_labels_retino, group))
        print('Region %s has %d voxels. Includes subregions:'%(ret_group_names[gi],n_this_region))
        inds = np.where(np.isin(ret[0],group))[0]
        print(list(np.array(ret[1])[inds]))
        ret_vox_total = ret_vox_total + n_this_region

    assert(np.sum(roi_labels_retino>0)==ret_vox_total)
    print('\n')
    categ_vox_total = 0
    categ_group_names = list(np.concatenate((np.array(face[1]), np.array(place[1])), axis=0))
    categ_group_inds = [[ii+26] for ii in range(len(categ_group_names))]

    # checking these grouping labels to make sure we have them correct (print which subregions go to which label)
    for gi, group in enumerate(categ_group_inds):
        n_this_region = np.sum(np.isin(roi_labels_categ, group))
        print('Region %s has %d voxels.'%(categ_group_names[gi],n_this_region))   
        categ_vox_total = categ_vox_total + n_this_region

    assert(np.sum(roi_labels_categ>0)==categ_vox_total)
    
    return voxel_mask, voxel_idx, [roi_labels_retino, roi_labels_categ], ncsnr_full, np.array(brain_nii_shape)

In [133]:
# Print all areas that are overlapping by any amount

print('Kastner areas overlap w pRF areas:')
print_overlap(kast_labels_full, prf_labels_full, ret, ret)
print('Kastner areas overlap w face areas:')
print_overlap(kast_labels_full, face_labels_full, ret, face)
print('Kastner areas overlap w place areas:')
print_overlap(kast_labels_full, place_labels_full, ret, place)
print('PRF areas overlap w face areas:')
print_overlap(prf_labels_full, face_labels_full, ret, face)
print('PRF areas overlap w place areas:')
print_overlap(prf_labels_full, place_labels_full, ret, place)
print('Face areas overlap w place areas:')
print_overlap(face_labels_full, place_labels_full, face, place)

Kastner areas overlap w pRF areas:
V1v and V1v:
 490 vox of overlap
V1v and V1d:
 83 vox of overlap
V1v and V2v:
 70 vox of overlap
V1d and V1v:
 19 vox of overlap
V1d and V1d:
 541 vox of overlap
V1d and V2d:
 29 vox of overlap
V2v and V1v:
 23 vox of overlap
V2v and V2v:
 506 vox of overlap
V2v and V3v:
 20 vox of overlap
V2d and V1d:
 73 vox of overlap
V2d and V2d:
 390 vox of overlap
V2d and V3d:
 7 vox of overlap
V3v and V2v:
 52 vox of overlap
V3v and V3v:
 402 vox of overlap
V3v and hV4:
 35 vox of overlap
V3d and V2d:
 101 vox of overlap
V3d and V3d:
 373 vox of overlap
hV4 and V3v:
 83 vox of overlap
hV4 and hV4:
 356 vox of overlap
VO1 and hV4:
 47 vox of overlap
LO1 and V3d:
 48 vox of overlap
V3B and V3d:
 27 vox of overlap
V3A and V3d:
 19 vox of overlap
Kastner areas overlap w face areas:
hV4 and OFA:
 22 vox of overlap
hV4 and FFA-1:
 6 vox of overlap
VO1 and FFA-1:
 20 vox of overlap
VO2 and FFA-1:
 2 vox of overlap
TO1 and OFA:
 3 vox of overlap
LO2 and OFA:
 9 vox of 